![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Mean Reversion
A number of financial signals can be thought to be mean-reverting (i.e. the spread between two assets prices). There are a number of ways to test for mean-reversion, but in this example we will assume it holds.


We use the research notebook to calculate the spread between an asset's price and its historical mean (past 30 days). Then, we calculate the standard deviation and determine which assets
have moved more than one standard deviation below their average price. The theory is that they are due to revert back towards the mean, and so we can take advantage of this by taking 
long positions.

In [1]:
// QuantBook C# Research Environment
// For more information see https://www.quantconnect.com/docs/research/overview
#load "../QuantConnect.csx"

// Initialize QuantBook
var qb = new QuantBook();

var assets = new List<string>() {"SHY", "TLT", "SHV", "TLH", "EDV", "BIL",
                  "SPTL", "TBT", "TMF", "TMV", "TBF", "VGSH", "VGIT",
                  "VGLT", "SCHO", "SCHR", "SPTS", "GOVT"};

// Subscribed to data for given assets
foreach(var ticker in assets){
    qb.AddEquity(ticker, Resolution.Minute);
}

// Request past 30 days of historical price data for given assets
var history = qb.History(qb.Securities.Keys, 30, Resolution.Daily);


var closes = new Dictionary<Symbol, List<Decimal>>();

// extract close prices for each Symbol from Slice data
foreach(var slice in history){
    foreach(var symbol in slice.Keys){
        if(!closes.ContainsKey(symbol)){
            closes.Add(symbol, new List<Decimal>());
        }
        closes[symbol].Add(slice.Bars[symbol].Close);
    }
}


In [2]:
var means = new Dictionary<Symbol, Decimal>();

// Calculate mean price over 30 day period for each symbol
foreach(var symbol in closes.Keys){
    if(!means.ContainsKey(symbol)){
        var avg = closes[symbol].Average();
        means.Add(symbol, avg);
    }
}

var std = new Dictionary<Symbol, Decimal>();

// Calculate standard deviation of prices for each symbol
foreach(var symbol in closes.Keys){
    if(!std.ContainsKey(symbol)){
        var average = means[symbol];
        var sumOfSquaresOfDifferences = closes[symbol].Select(val => (val - average) * (val - average)).Sum();
        var sd = (Decimal)Math.Sqrt((double) (sumOfSquaresOfDifferences / closes[symbol].Count()));

        std.Add(symbol, sd);
    }
}

In [3]:
var selected = new List<Symbol>();

// Select symbols which are 1 standard deviation below their 30 day mean
foreach(var symbol in closes.Keys){
    var close_values = closes[symbol];
    var last_close = close_values[close_values.Count - 1];
    var lower_bound = means[symbol] - std[symbol];
    
    if(last_close < lower_bound){
        selected.Add(symbol);
    }
}

selected.ForEach(x => {Console.WriteLine(x);});